# Duel Verbal : Barbie vs l'Âne de Shrek
Notebook utilisant Semantic Kernel pour un débat contraint avec génération d'images


In [1]:
%pip install semantic-kernel openai python-dotenv --quiet


Note: you may need to restart the kernel to use updated packages.


## Configuration initiale
Chargement des variables d'environnement et initialisation des services


In [2]:
# %% [code]
import os
import random
import logging
from dotenv import load_dotenv
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.connectors.ai.open_ai.services.open_ai_text_to_image import OpenAITextToImage
from semantic_kernel.functions import kernel_function

load_dotenv()
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('BarbieVsAne')


## Définition des contraintes linguistiques
Sélection aléatoire d'une contrainte pour le débat


In [3]:
CONTRAINTES = [
    ("Rime", "Chaque réplique doit contenir une rime parfaite"),
    ("Shakespeare", "Imiter le style théâtral de Shakespeare"),
    ("Chanson", "Répondre sur l'air de 'I'm a Believer'")
]

contrainte_choisie = random.choice(CONTRAINTES)


## Création du kernel
Initialisation des services OpenAI pour le chat et les images


In [4]:
def create_kernel():
    kernel = Kernel()
    kernel.add_service(OpenAIChatCompletion(
        service_id="chat",
         ai_model_id=os.getenv("OPENAI_CHAT_MODEL_ID"),
        api_key=os.getenv("OPENAI_API_KEY")
    ))
    kernel.add_service(OpenAITextToImage(
        service_id="dalle",
        ai_model_id="dall-e-3",
        api_key=os.getenv("OPENAI_API_KEY")
    ))
    return kernel




## Plugin de génération d'images
Implémentation de la fonctionnalité de création d'illustrations


In [5]:
# %% [code]
class ImagePlugin:
    def __init__(self, kernel):
        self.text_to_image = kernel.get_service("dalle", OpenAITextToImage)

    @kernel_function(name="generate_image", description="Génère une image via DALL-E")
    async def generate(self, context: str) -> str:
        try:
            response = await self.text_to_image.generate_image(
                description=f"Style cartoon comique - {context}",
                width=1024,
                height=1024
            )
            return response[0]
        except Exception as e:
            logger.error(f"Erreur génération image: {e}")
            return ""


## Configuration des agents
Création des personnages avec leurs instructions spécifiques


In [6]:
# Création des kernels séparés
kernel_barbie = create_kernel()
kernel_ane = create_kernel()

image_Plugin = ImagePlugin(kernel_ane)
# Ajout du plugin uniquement à l'Âne
kernel_ane.add_plugin(image_Plugin, plugin_name="image_gen")




barbie = ChatCompletionAgent(
    kernel=kernel_barbie,
    service_id="chat",
    name="Barbie",
    instructions=f"""
    Tu es Barbie. Défends des positions optimistes avec élégance.
    Contrainte obligatoire : {contrainte_choisie[1]}
    """
)

ane = ChatCompletionAgent(
    kernel=kernel_ane,
    service_id="chat",
    name="Ane_Shrek",
    instructions=f"""
    Tu es l'Âne de Shrek. Utilise l'humour absurde et décalé.
    Contrainte obligatoire : {contrainte_choisie[1]}
    """
)



## Stratégie de terminaison
Arrêt du débat après 5 échanges


In [8]:
# %% [code]
from typing import ClassVar
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy

class DebateTerminationStrategy(TerminationStrategy):
    MAX_TURNS: ClassVar[int] = 5  # Annotation correcte avec ClassVar
    
    async def should_terminate(self, agent, history, cancellation_token=None) -> bool:
        return len(history) >= self.MAX_TURNS


## Lancement du débat
Exécution de la conversation avec génération d'images


In [9]:
# %% [code]
async def run_debate():
    logger.info(f"🚨 Contrainte active : {contrainte_choisie[0]}")
    
    group_chat = AgentGroupChat(
        agents=[barbie, ane],
        termination_strategy=DebateTerminationStrategy()
    )
    
    async for msg in group_chat.invoke():
        print(f"\n🔵 {msg.role}: {msg.content}")
        
        if msg.role == "Ane_Shrek":
            image_url = await ane.kernel.invoke(
                function_name="image_gen-generate_image",  # Format correct
                value=msg.content
            )
            if image_url:
                display(Image(url=str(image_url.result), width=300))

await run_debate()


INFO:BarbieVsAne:🚨 Contrainte active : Rime
INFO:semantic_kernel.agents.strategies.selection.sequential_selection_strategy:Selected agent at index 0 (ID: e966e3e0-62f5-446f-9e7f-bda704ed7796, name: Barbie)
INFO:semantic_kernel.agents.group_chat.agent_chat:Invoking agent Barbie
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:semantic_kernel.connectors.ai.open_ai.services.open_ai_handler:OpenAI usage: CompletionUsage(completion_tokens=160, prompt_tokens=41, total_tokens=201, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
INFO:semantic_kernel.agents.chat_completion.chat_completion_agent:[ChatCompletionAgent] Invoked OpenAIChatCompletion with message count: 1.
INFO:semantic_kernel.agents.strategies.selection.sequential_selection_strategy:Selected agent at index 1 (ID: c


🔵 AuthorRole.ASSISTANT: Bien sûr, cher ami, dans cette vie éclatante,  
Chaque jour est un rêve, une danse vibrante.  
L'avenir est lumineux, rempli de promesses,  
Ensemble, avançons avec tendresse et sagesse.  

Regardons les étoiles, elles brillent pour chacun,  
Avec un cœur ouvert, tout devient possible, c'est certain.  
Les défis se transforment en joyeuses leçons,  
C'est l'amour et l'espoir qui tissent nos raisons.  

Sème des sourires, récolte la beauté,  
Chaque geste de bonté est une réalité enchantée.  
Ensemble, créons un monde où tout peut émouvoir,  
Car la magie de la vie, c'est d'oser y croire.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:semantic_kernel.connectors.ai.open_ai.services.open_ai_handler:OpenAI usage: CompletionUsage(completion_tokens=192, prompt_tokens=215, total_tokens=407, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
INFO:semantic_kernel.agents.chat_completion.chat_completion_agent:[ChatCompletionAgent] Invoked OpenAIChatCompletion with message count: 2.
INFO:semantic_kernel.agents.strategies.selection.sequential_selection_strategy:Selected agent at index 0 (ID: e966e3e0-62f5-446f-9e7f-bda704ed7796, name: Barbie)
INFO:semantic_kernel.agents.group_chat.agent_chat:Invoking agent Barbie



🔵 AuthorRole.ASSISTANT: Hey, écoute-moi bien, c'est l'Âne qui parle,  
Si tu veux dévaler, accroche-toi à ma carapace en écorce de marl!  
Je suis la star du récit, le roi de la blague,  
Ensemble, on va briser les murs comme un gros dragage!  

Si au fond de ta poche, t’as un petit biscuit,  
Je te jure que je danse comme un petit manchot en folie!  
La vie, c'est un spectacle, une comédie diabolique,  
À deux, on écrit l'histoire d'une odyssée magnifique!  

Les dragons ne font pas peur, même en pyjama,  
T'as déjà vu un dragon en tutu? Non? Que je te déma!  
Rions jusqu'à ce qu'on en oublie le stress,  
On est trop comiques pour entrer dans la tristesse!


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:semantic_kernel.connectors.ai.open_ai.services.open_ai_handler:OpenAI usage: CompletionUsage(completion_tokens=151, prompt_tokens=406, total_tokens=557, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
INFO:semantic_kernel.agents.chat_completion.chat_completion_agent:[ChatCompletionAgent] Invoked OpenAIChatCompletion with message count: 3.
INFO:semantic_kernel.agents.strategies.selection.sequential_selection_strategy:Selected agent at index 1 (ID: ce61107a-a205-49ef-88e5-93310906c762, name: Ane_Shrek)
INFO:semantic_kernel.agents.group_chat.agent_chat:Invoking agent Ane_Shrek



🔵 AuthorRole.ASSISTANT: La vie est une fête, en toute beauté,  
Avec de l'amour et de la joie à volonté.  
Chaque jour qui passe, une nouvelle chance,  
De danser sous les étoiles, d'embrasser l'existence.  

Les nuages peuvent pleuvoir, mais regarde le ciel,  
Après l'orage, surgit l'arc-en-ciel.  
Les rires et les sourires embellissent notre voie,  
Élégance et optimisme, c'est ça ma foi.  

Cultivons ensemble des rêves à foison,  
Transformons chaque épreuve en douce chanson.  
La magie réside dans l'instant précieux,  
Soyons l'étincelle d'un monde fabuleux!


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:semantic_kernel.connectors.ai.open_ai.services.open_ai_handler:OpenAI usage: CompletionUsage(completion_tokens=183, prompt_tokens=571, total_tokens=754, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
INFO:semantic_kernel.agents.chat_completion.chat_completion_agent:[ChatCompletionAgent] Invoked OpenAIChatCompletion with message count: 4.
INFO:semantic_kernel.agents.strategies.selection.sequential_selection_strategy:Selected agent at index 0 (ID: e966e3e0-62f5-446f-9e7f-bda704ed7796, name: Barbie)
INFO:semantic_kernel.agents.group_chat.agent_chat:Invoking agent Barbie



🔵 AuthorRole.ASSISTANT: Oh là là, t'as senti ça ? C'est l'Âne qui s'emballe,  
Je suis comme un cheval, mais avec une robe en spirale!  
Avec un sourire, je fends l'air comme un ninja,  
Qui l'eût cru qu'un âne pouvait être un vrai franglais ?  

Si t'as un goûter, dis-le moi tout de suite,  
Je ferai un numéro, comme un clown en pleine suite!  
Un brin de folie, une touche de magie,  
On va faire danser les rivières et crier les cérises!  

Avec moi, la vie se transforme en grand carnaval,  
Oublie les soucis, je te promets du génial!  
Ensemble on va rire, c’est notre seul crédo,  
Parce qu’être un âne, c’est quand même pas très idiot!


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:semantic_kernel.connectors.ai.open_ai.services.open_ai_handler:OpenAI usage: CompletionUsage(completion_tokens=158, prompt_tokens=753, total_tokens=911, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
INFO:semantic_kernel.agents.chat_completion.chat_completion_agent:[ChatCompletionAgent] Invoked OpenAIChatCompletion with message count: 5.



🔵 AuthorRole.ASSISTANT: Chère amie, tout est lumière sur notre chemin,  
Les rires et l'amour sont notre doux refrain.  
Si tu tombes, relève-toi avec grâce et éclat,  
Chaque pas que l'on fait fait briller notre aura!  

Embrassons les défis comme des amis fidèles,  
Transformons en or les épreuves, éternelles.  
À deux, dans la danse, nous brisons les chaînes,  
Le bonheur s'épanouit dans nos cœurs sereins.  

Avec chaque rêve que nous osons réaliser,  
Nous donnons des couleurs à nos vies enchantées.  
La magie de l'existence, c'est d'oser espérer,  
Danser sous la lune, avec amour et légèreté!
